In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from glob import glob
import pickle

from PIL import ImageColor
import matplotlib.colors

from utils import FEATURES_DIR, RESULTS_DIR, SUBJECTS, NUM_TEST_STIMULI
from analyses.ridge_regression_decoding import NUM_CV_SPLITS, DECODER_OUT_DIR, calc_rsa, calc_rsa_images, calc_rsa_captions, get_fmri_data, pairwise_accuracy
from notebook_utils import load_results_data, add_avg_subject, create_result_graph

In [ ]:
all_data = load_results_data()

MODEL_ORDER = ["random-flava", "vit-b-16", "vit-l-16", "resnet-18", "resnet-50", "resnet-152", "dino-base", "dino-large", "dino-giant",
               "bert-base-uncased", "bert-large-uncased", "llama2-7b", "llama2-13b", "mistral-7b", "mixtral-8x7b", "gpt2-small", "gpt2-medium", "gpt2-large", "gpt2-xl",
               "visualbert", "bridgetower-large", "clip", "flava", "imagebind", "lxmert", "vilt"]

all_data = all_data[all_data.model.isin(MODEL_ORDER)]
all_data

In [ ]:
all_data_avg = add_avg_subject(all_data)
all_data_avg = all_data_avg[all_data_avg.subject == "average"]

In [ ]:
# ref performance order:
#['random-flava', 'resnet-152', 'vit-b-16', 'vit-l-16', 'resnet-18', 'resnet-50', 'dino-giant', 'dino-large', 'dino-base', 'bert-large-uncased', 'bert-base-uncased',
# 'llama2-7b', 'mixtral-8x7b', 'mistral-7b', 'llama2-13b', 'gpt2-small', 'gpt2-medium', 'gpt2-xl', 'gpt2-large', 'visualbert', 'bridgetower-large', 'clip', 'flava', 'imagebind', 'lxmert', 'vilt']
FEAT_OPTIONS = ["vision", "lang", "vision+lang", "matched"]

def calc_model_feat_order(data):
    all_model_feats = data.model_feat.unique()
    all_models = data.model.unique()
    for model in all_models:
        if model not in MODEL_ORDER:
            raise RuntimeError(f"Model missing in order: {model}")
    model_feat_order = []
    for model in MODEL_ORDER:
        for feats in FEAT_OPTIONS:
            model_feat = f"{model}_{feats}"
            if model_feat in all_model_feats:
                model_feat_order.append(model_feat)

    return model_feat_order

for model in MODEL_ORDER:
    print(model, end=" ")

In [ ]:
distance_metric = 'cosine'
METRICS = [f'acc_{distance_metric}', f'acc_{distance_metric}_captions', f'acc_{distance_metric}_images']


## Modality-agnostic decoding vs. modality-specific decoding

In [ ]:
multimodal_models = all_data_avg[all_data_avg.features == "vision+lang"].model.unique().tolist()
MODEL_FEAT_MULTIMODAL_SINGLE_MODALITY = [m+'_lang' for m in multimodal_models] + [m+'_vision' for m in multimodal_models]
MODEL_FEAT_MULTIMODAL_SINGLE_MODALITY += [m+'_matched' for m in multimodal_models]

vision_models = [m for m in all_data_avg[all_data_avg.features == "vision"].model.unique() if len(all_data_avg[all_data_avg.model == m].features.unique()) == 1]
for m in multimodal_models:
    print(m, end=" ")

### Model performance ranking

In [ ]:
dp = all_data_avg

MODEL_FEATS_EXCLUDED = ["bridgetower-large_multi", "random-flava_vision", "random-flava_lang"] + MODEL_FEAT_MULTIMODAL_SINGLE_MODALITY
dp = dp[~dp.model_feat.isin(MODEL_FEATS_EXCLUDED)].copy()
dp = dp[dp["mask"] == "whole_brain"]

model_order = ['random-flava']
for features in ["vision", "lang", "vision+lang"]:
    model_scores = []
    data_feats = dp[(dp.features == features) & (dp.model != 'random-flava')]
    models = data_feats.model.unique()
    for model in models:
        dp_model = dp[(dp.model == model) & (dp.training_mode == 'modality-agnostic')]

        dp_model = dp_model[(dp_model.vision_features == 'visual_feature_mean')]
        
        dp_model_caps = dp_model[dp_model.metric == "pairwise_acc_captions"]
        assert len(dp_model_caps) == 6, f"{model}: {len(dp_model_caps)} entries"
        # print(f"pairwise_acc_captions: {dp_model_caps.value.mean():.2f}")
        dp_model_imgs = dp_model[dp_model.metric == "pairwise_acc_images"]
        assert len(dp_model_imgs) == 6, len(dp_model_imgs)
        # print(f"pairwise_acc_images: {dp_model_imgs.value.mean():.2f}")
        avg = np.mean([dp_model_caps.value.mean(), dp_model_imgs.value.mean()])
        print(f"{model} pairwise_acc mean : {dp_model[dp_model.metric == 'pairwise_acc_mean'].value.mean():.3f} std: {dp_model[dp_model.metric == 'pairwise_acc_mean'].value.std():.3f}")
        model_scores.append(avg)
        
    model_order.extend([model for _, model in sorted(zip(model_scores, models), key=lambda pair: pair[0])])
print(model_order)



In [ ]:
sns.set(font_scale=1.6)
TRAIN_MODE_ORDER = ["images", "captions", "modality-agnostic"]
FEAT_ORDER = ["vision models", "language models", "multimodal models"]

MODEL_FEATS_EXCLUDED = ["bridgetower-large_multi", "random-flava_vision", "random-flava_lang"] + MODEL_FEAT_MULTIMODAL_SINGLE_MODALITY
data_to_plot = all_data_avg[~all_data_avg.model_feat.isin(MODEL_FEATS_EXCLUDED)].copy()

data_to_plot = data_to_plot[data_to_plot.vision_features == 'visual_feature_mean']

data_to_plot["features"] = data_to_plot.features.replace({"vision": "vision models", "lang": "language models", "vision+lang": "multimodal models"})

data_to_plot = data_to_plot[data_to_plot["mask"] == "whole_brain"]
model_feat_order = calc_model_feat_order(data_to_plot)

metrics_order = ["pairwise_acc_captions", "pairwise_acc_images", "pairwise_acc_mean"]
figure, lgd = create_result_graph(data_to_plot, model_feat_order, metrics=metrics_order, row_order=metrics_order, hue_order=FEAT_ORDER, ylim=(0.5, 1),
                             legend_bbox=(0.06,0.99), height=4.5, legend_title="Modality-agnostic decoders projecting into feature space of")
plt.savefig(os.path.join(RESULTS_DIR, f"features_comparison_pairwise_acc.png"), bbox_extra_artists=(lgd,), bbox_inches='tight', pad_inches=0, dpi=300)


## V, L, or multimodal feats for multimodal models

In [ ]:
feat_legend = {"vision": "vision feature space", "lang": "language feature space", "vision+lang": "multimodal feature space: concat", "matched": "matched feature space"}
feat_order = ["vision", "lang", "vision+lang", "matched"]
feat_order = [feat_legend[feat] for feat in feat_order]

model_feats_exluded = ["bridgetower-large_multi", "random-flava_vision", "random-flava_lang", "random-flava_vision+lang"]
data_to_plot = all_data_avg[~all_data_avg.model_feat.isin(model_feats_exluded)].copy()

data_to_plot = data_to_plot[data_to_plot.model.isin(multimodal_models)]
data_to_plot = data_to_plot[data_to_plot.vision_features == 'visual_feature_mean']

data_to_plot["features"] = data_to_plot.features.replace(feat_legend)

data_to_plot = data_to_plot[data_to_plot["mask"] == "whole_brain"]
model_feat_order = calc_model_feat_order(data_to_plot)

metrics_order = ["pairwise_acc_captions", "pairwise_acc_images", "pairwise_acc_mean"]
figure, lgd = create_result_graph(data_to_plot, model_feat_order, metrics=metrics_order, row_order=metrics_order, hue_order=feat_order, ylim=(0.5, 1),
                             legend_bbox=(0.06,1.01), height=4.5, legend_title="Modality-agnostic decoders projecting into")
plt.savefig(os.path.join(RESULTS_DIR, f"features_comparison_pairwise_acc_multimodal_models.png"), bbox_extra_artists=(lgd,), bbox_inches='tight', pad_inches=0, dpi=300)

## CLS or mean feats for vision models

In [ ]:
model_feats_exluded = ["bridgetower-large_multi", "random-flava_vision", "random-flava_lang", "random-flava_vision+lang"]
data_to_plot = all_data_avg[~all_data_avg.model_feat.isin(model_feats_exluded)].copy()

vision_models_both_options = [m for m in vision_models if len(data_to_plot[data_to_plot.model == m].vision_features.unique()) == 2]
data_to_plot = data_to_plot[data_to_plot.model.isin(vision_models_both_options)]

vision_feat_order = ["Mean vision features", "CLS token vision features"]
data_to_plot["vision_features"] = data_to_plot.vision_features.replace({"visual_feature_mean": "Mean vision features", "visual_feature_cls": "CLS token vision features"})

data_to_plot = data_to_plot[data_to_plot.training_mode == "modality-agnostic"]

data_to_plot = data_to_plot[data_to_plot["mask"] == "whole_brain"]
model_feat_order = calc_model_feat_order(data_to_plot)

metrics_order = ["pairwise_acc_captions", "pairwise_acc_images", "pairwise_acc_mean"]
figure, lgd = create_result_graph(data_to_plot, model_feat_order, metrics=metrics_order, row_order=metrics_order, hue_variable="vision_features", hue_order=vision_feat_order, ylim=(0.5, 1),
                             legend_bbox=(0.06,1.0), height=4.5, legend_title="Modality-agnostic decoders projecting into", plot_modality_specific=False)
plt.savefig(os.path.join(RESULTS_DIR, f"features_comparison_pairwise_acc_vision_models.png"), bbox_extra_artists=(lgd,), bbox_inches='tight', pad_inches=0, dpi=300)

## ROI-based decoding

In [ ]:
MASK_ORDER = ["high-level visual ROI", "low-level visual ROI", "language ROI"]
# MASK_ORDER = ["high-level visual ROI", "low-level visual ROI", "language ROI", "functional_Language", "functional_Visual1", "functional_Visual2", "functional_Default", "functional_Visual1_Visual2"]

MASK_PALETTE = sns.color_palette('Set2')[3:3+len(MASK_ORDER)][::-1]

# MODEL_FEATS_INCLUDED = ["resnet-50_vision", "dino-giant_vision", "gpt2-large_lang", "llama2-13b_lang", "flava_vision+lang"]
# data_all_masks = all_data_avg[all_data_avg.model_feat.isin(MODEL_FEATS_INCLUDED)].copy()

MODEL_FEATS_EXCLUDED = ["bridgetower-large_multi", "random-flava_vision", "random-flava_lang"] + MODEL_FEAT_MULTIMODAL_SINGLE_MODALITY
data_all_masks = all_data_avg[~all_data_avg.model_feat.isin(MODEL_FEATS_EXCLUDED)].copy()

data_all_masks = data_all_masks[data_all_masks.vision_features == 'visual_feature_mean']


data_all_masks["mask"] = data_all_masks["mask"].replace({"anatomical_visual_low_level": "low-level visual ROI", "anatomical_lang": "language ROI", "anatomical_visual_high_level": "high-level visual ROI"})

data_all_masks = data_all_masks[data_all_masks["mask"].isin(MASK_ORDER)].copy()

model_feat_order = calc_model_feat_order(data_all_masks)

metrics_order = ["pairwise_acc_captions", "pairwise_acc_images", "pairwise_acc_mean"]

dodge = 0.47
# dodge = 0.6
figure, lgd = create_result_graph(data_all_masks, model_feat_order, metrics=metrics_order, row_order=metrics_order, hue_variable="mask", hue_order=MASK_ORDER, palette=MASK_PALETTE, ylim=(0.5, 1),
                                  legend_title="Modality-agnostic decoders trained on fMRI data from", dodge=dodge, legend_bbox=(0.06,0.99))

colors_bg = sns.color_palette('Set2')[:3]
for i in range(len(figure.axes)):
    figure.axes[i, 0].axvspan(-0.5, 0.5, facecolor=colors_bg[2], alpha=0.2, zorder=-100)
    figure.axes[i, 0].axvspan(0.5, 8.5, facecolor=colors_bg[0], alpha=0.2, zorder=-100)
    figure.axes[i, 0].axvspan(8.5, 18.5, facecolor=colors_bg[1], alpha=0.2, zorder=-100)
    figure.axes[i, 0].axvspan(18.5, 25.5, facecolor=colors_bg[2], alpha=0.2, zorder=-100)
plt.xlim((-0.5, 25.5))


# plt.subplots_adjust(top=0.98, bottom=0.05, hspace=0)
plt.savefig(os.path.join(RESULTS_DIR, f"roi_comparison_pairwise_acc.png"), bbox_extra_artists=(lgd,), bbox_inches='tight', pad_inches=0, dpi=300)

In [ ]:
n_voxels_data = data_all_masks[~data_all_masks.num_voxels.isna()]
n_voxels_data = {mask: n_voxels_data[n_voxels_data["mask"] == mask].num_voxels.mean() for mask in n_voxels_data["mask"].unique()}
# n_voxels_data.update({
#     # "whole_brain": 214739,
#     "visual_high_level": 14698,
#     "visual_low_level": 13955
# })
print(n_voxels_data)
# sns.barplot(data=n_voxels_data)
# # plt.yscale("log")
# plt.xticks(rotation = 80)
# plt.ylabel("num voxels")
# plt.title("Number of voxels for each mask (whole brain: 214,739)")
# plt.tight_layout()
# plt.savefig(os.path.join(RESULTS_DIR, f"num_voxels.png"), dpi=300)

## Per-subject results

In [ ]:
FEAT_ORDER = ["vision", "lang", "vision+lang"]
FEAT_PALETTE = sns.color_palette('Set2')[:3]

def create_result_graph_all_subjs(data, model_feat_order, metrics=["pairwise_acc_captions", "pairwise_acc_images"], hue_variable="features", hue_order=FEAT_ORDER, ylim=None,
                        legend_title="Modality-agnostic decoders based on features from", palette=FEAT_PALETTE, dodge=False, noise_ceilings=None, plot_modality_specific=True,
                       row_variable="metric", col_variable=None):
    data_training_mode_full = data[data.training_mode == "modality-agnostic"]


    assert len(data_training_mode_full[(data_training_mode_full.model_feat == 'gpt2-large_lang') & (data_training_mode_full.metric == metrics[0]) & (data_training_mode_full[hue_variable] == data_training_mode_full[hue_variable].values[0])]) == 6

    catplot_g, data_plotted, lgd = plot_metric_catplot(data_training_mode_full, order=model_feat_order, metrics=metrics, x_variable="model_feat", legend_title=legend_title, aspect=2, legend_bbox=(0,1.05), rotation=89, cut_labels=False,
                                                  hue_variable=hue_variable, row_variable=row_variable, col_variable=col_variable, hue_order=hue_order, palette=palette, ylim=ylim, noise_ceilings=noise_ceilings)

    if plot_modality_specific:
        for i, subj in zip(range(6), SUBJECTS):
            data_subj = data[data.subject == subj]
            data_training_mode_captions = data_subj[data_subj.training_mode == "captions"]
            data_training_mode_images = data_subj[data_subj.training_mode == "images"]
            _ = plot_metric(data_training_mode_captions, kind="point", order=model_feat_order, metric=metrics[0], x_variable="model_feat", dodge=dodge,
                                          hue_variable=hue_variable, hue_order=hue_order, palette=[(0, 0, 0)], axis=catplot_g.axes[i, 0], marker="o", plot_legend=False, ylim=ylim)
            g, _ = plot_metric(data_training_mode_images, kind="point", order=model_feat_order, metric=metrics[0], x_variable="model_feat", dodge=dodge,
                                          hue_variable=hue_variable, hue_order=hue_order, palette=[(0, 0, 0)], axis=catplot_g.axes[i, 0], marker="x", plot_legend=False, ylim=ylim)
            
            _ = plot_metric(data_training_mode_captions, kind="point", order=model_feat_order, metric=metrics[1], x_variable="model_feat", dodge=dodge,
                                          hue_variable=hue_variable, hue_order=hue_order, palette=[(0, 0, 0)], axis=catplot_g.axes[i, 1], marker="o", plot_legend=False, ylim=ylim)
            _ = plot_metric(data_training_mode_images, kind="point", order=model_feat_order, metric=metrics[1], x_variable="model_feat", dodge=dodge,
                                          hue_variable=hue_variable, hue_order=hue_order, palette=[(0, 0, 0)], axis=catplot_g.axes[i, 1], marker="x", plot_legend=False, ylim=ylim)
        
        handles, labels = g.get_legend_handles_labels()
        new_labels = ["captions", "images"]
        new_handles = [handles[0], handles[-1]]
        catplot_g.fig.legend(handles=new_handles, labels=new_labels, ncol=2, title="Modality-specific decoders trained on", loc='upper right')

        for i in range(6):
            catplot_g.axes[i,0].set_title(f"subject {i+1} | captions", fontsize=25)
            catplot_g.axes[i,1].set_title(f"subject {i+1} | images", fontsize=25)

            catplot_g.axes[i,0].set_ylabel('pairwise accuracy')

    return catplot_g, lgd


In [ ]:
TRAIN_MODE_ORDER = ["images", "captions", "modality-agnostic"]
FEAT_ORDER = ["vision model", "language model", "multimodal model"]

MODEL_FEATS_EXCLUDED = ["bridgetower-large_multi", "random-flava_vision", "random-flava_lang"] + MODEL_FEAT_MULTIMODAL_SINGLE_MODALITY
data_to_plot = all_data[~all_data.model_feat.isin(MODEL_FEATS_EXCLUDED)].copy()

data_to_plot = data_to_plot[data_to_plot.vision_features == 'visual_feature_mean']

data_to_plot["features"] = data_to_plot.features.replace({"vision": "vision model", "lang": "language model", "vision+lang": "multimodal model"})

data_to_plot = data_to_plot[data_to_plot["mask"] == "whole_brain"]
model_feat_order = calc_model_feat_order(data_to_plot)

figure, lgd = create_result_graph_all_subjs(data_to_plot, model_feat_order, metrics=["pairwise_acc_captions", "pairwise_acc_images"], hue_order=FEAT_ORDER, ylim=(0.5, 1), row_variable="subject", col_variable="metric")
plt.savefig(os.path.join(RESULTS_DIR, f"features_comparison_pairwise_acc_per_subject.png"), dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight', pad_inches=0)